In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import os

In [2]:
train=pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
train

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0
1,1,2013-01-01,1,BABY CARE,0.000,0
2,2,2013-01-01,1,BEAUTY,0.000,0
3,3,2013-01-01,1,BEVERAGES,0.000,0
4,4,2013-01-01,1,BOOKS,0.000,0
...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8


In [3]:
holiday=pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv')
oil=pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv')

In [4]:
train=pd.merge(train, holiday, on='date', how='left')
train=pd.merge(train, oil, on='date', how='left')

In [5]:
stores=pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')

In [6]:
train=pd.merge(train, stores, on='store_nbr', how='left')

In [7]:
transactions=pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')

In [8]:
train=pd.merge(train, transactions, on=["date", "store_nbr"] , how='left')

In [9]:
train.isnull().sum()

id                    0
date                  0
store_nbr             0
family                0
sales                 0
onpromotion           0
type_x          2551824
locale          2551824
locale_name     2551824
description     2551824
transferred     2551824
dcoilwtico       955152
city                  0
state                 0
type_y                0
cluster               0
transactions     249117
dtype: int64

In [10]:
train['date'] = pd.to_datetime(train['date'])
train['day'] = train['date'].dt.day
train['month'] = train['date'].dt.month
train['year'] = train['date'].dt.year
train['weekday_num'] = train['date'].dt.weekday
train['day_of_year'] = train['date'].dt.dayofyear
train['quarter'] = train['date'].dt.quarter
train['is_leap_year'] = train['date'].dt.is_leap_year
train['is_weekday'] = train['date'].dt.weekday < 5
train['is_weekend'] = train['date'].dt.weekday >= 5

In [11]:
train['family'].unique()

array(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
       'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
       'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
       'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
       'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
       'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
       'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
       'SEAFOOD'], dtype=object)

In [12]:
train.head()

,id,date,store_nbr,family,sales,onpromotion,type_x,locale,locale_name,description,...,transactions,day,month,year,weekday_num,day_of_year,quarter,is_leap_year,is_weekday,is_weekend
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Holiday,National,Ecuador,Primer dia del ano,...,NaN,1,1,2013,1,1,1,False,True,False
1,1,2013-01-01,1,BABY CARE,0.0,0,Holiday,National,Ecuador,Primer dia del ano,...,NaN,1,1,2013,1,1,1,False,True,False
2,2,2013-01-01,1,BEAUTY,0.0,0,Holiday,National,Ecuador,Primer dia del ano,...,NaN,1,1,2013,1,1,1,False,True,False
3,3,2013-01-01,1,BEVERAGES,0.0,0,Holiday,National,Ecuador,Primer dia del ano,...,NaN,1,1,2013,1,1,1,False,True,False
4,4,2013-01-01,1,BOOKS,0.0,0,Holiday,National,Ecuador,Primer dia del ano,...,NaN,1,1,2013,1,1,1,False,True,False


In [13]:
train['type_x'].unique()

array(['Holiday', nan, 'Work Day', 'Additional', 'Event', 'Transfer',
       'Bridge'], dtype=object)

In [14]:
train['locale'].unique()

array(['National', nan, 'Local', 'Regional'], dtype=object)

In [15]:
train['description'].unique()

array(['Primer dia del ano', nan, 'Recupero puente Navidad',
       'Recupero puente primer dia del ano', 'Carnaval',
       'Fundacion de Manta', 'Provincializacion de Cotopaxi',
       'Fundacion de Cuenca', 'Cantonizacion de Libertad',
       'Cantonizacion de Riobamba', 'Viernes Santo', 'Dia del Trabajo',
       'Dia de la Madre-1', 'Cantonizacion del Puyo', 'Dia de la Madre',
       'Batalla de Pichincha', 'Cantonizacion de Guaranda',
       'Provincializacion de Imbabura', 'Fundacion de Machala',
       'Cantonizacion de Latacunga', 'Cantonizacion de El Carmen',
       'Fundacion de Santo Domingo', 'Cantonizacion de Cayambe',
       'Fundacion de Guayaquil-1', 'Fundacion de Guayaquil',
       'Fundacion de Esmeraldas', 'Primer Grito de Independencia',
       'Fundacion de Riobamba', 'Fundacion de Ambato',
       'Fundacion de Ibarra', 'Cantonizacion de Quevedo',
       'Independencia de Guayaquil',
       'Traslado Independencia de Guayaquil', 'Dia de Difuntos',
       'Independe

In [16]:
train['city'].unique()

array(['Quito', 'Cayambe', 'Latacunga', 'Riobamba', 'Ibarra',
       'Santo Domingo', 'Guaranda', 'Puyo', 'Ambato', 'Guayaquil',
       'Salinas', 'Daule', 'Babahoyo', 'Quevedo', 'Playas', 'Libertad',
       'Cuenca', 'Loja', 'Machala', 'Esmeraldas', 'Manta', 'El Carmen'],
      dtype=object)

In [17]:
train['locale']=train['locale'].fillna('None')

In [18]:
train['type_x']=train['type_x'].fillna('None')

In [19]:
train['locale_name']=train['locale_name'].fillna('None')

In [20]:
train['transferred']=train['transferred'].fillna('None')

In [21]:
train['dcoilwtico']=train['dcoilwtico'].fillna(train['dcoilwtico'].mean())

In [22]:
train['transactions']=train['transactions'].fillna(train['transactions'].mean())

In [23]:
train['description']=train['description'].fillna('None')

In [24]:
train.isnull().sum()

id              0
date            0
store_nbr       0
family          0
sales           0
onpromotion     0
type_x          0
locale          0
locale_name     0
description     0
transferred     0
dcoilwtico      0
city            0
state           0
type_y          0
cluster         0
transactions    0
day             0
month           0
year            0
weekday_num     0
day_of_year     0
quarter         0
is_leap_year    0
is_weekday      0
is_weekend      0
dtype: int64

In [25]:
train['extracted_number'] = train['description'].str.extract(r'([+-]\d+)')

In [26]:
train['extracted_number']=train['extracted_number'].fillna(0)

In [27]:
train['sentence'] = train['description'].str.replace(r'[+-]\d+', '', regex=True).str.strip()
sorted(train['sentence'].unique())

['Batalla de Pichincha',
 'Black Friday',
 'Cantonizacion de Cayambe',
 'Cantonizacion de El Carmen',
 'Cantonizacion de Guaranda',
 'Cantonizacion de Latacunga',
 'Cantonizacion de Libertad',
 'Cantonizacion de Quevedo',
 'Cantonizacion de Riobamba',
 'Cantonizacion de Salinas',
 'Cantonizacion del Puyo',
 'Carnaval',
 'Cyber Monday',
 'Dia de Difuntos',
 'Dia de la Madre',
 'Dia del Trabajo',
 'Fundacion de Ambato',
 'Fundacion de Cuenca',
 'Fundacion de Esmeraldas',
 'Fundacion de Guayaquil',
 'Fundacion de Ibarra',
 'Fundacion de Loja',
 'Fundacion de Machala',
 'Fundacion de Manta',
 'Fundacion de Quito',
 'Fundacion de Riobamba',
 'Fundacion de Santo Domingo',
 'Inauguracion Mundial de futbol Brasil',
 'Independencia de Ambato',
 'Independencia de Cuenca',
 'Independencia de Guaranda',
 'Independencia de Guayaquil',
 'Independencia de Latacunga',
 'Mundial de futbol Brasil: Cuartos de Final',
 'Mundial de futbol Brasil: Ecuador-Francia',
 'Mundial de futbol Brasil: Ecuador-Hondur